### I repeated the below process couple of times to generate a large dataset.For generating data I took movies sound track and labelled the speech containing parts and stored the labelling in csv file having same name as that of audio track.

In [1]:
import numpy as np
import dask.array as da
import os
import warnings
import h5py
import librosa
warnings.filterwarnings("ignore")

### Extracting the portion from the audio track which contains speech which is well labelled.

In [ ]:
for name in ['track81','track82','track83']:
    annotations=pd.read_csv('./Custom/{filename}.csv'.format(filename=name),header=None)

    X_p=[]
    X_n=[]
    Y_p=[]
    Y_n=[]
    test=[]
    count=0
    #len(annotations)
    for i in range(len(annotations)):
        try:

            print("Pass:%d"%i)
            _,start,end,lb=annotations.iloc[i]
            start,end,lb=int(start),int(end),int(lb)

            for j in range(abs(round((start-end))+1)):
                if lb==1:
                    audio,sr=librosa.load('./Custom/{filename}.mp3'.format(filename=name),offset=start,duration=1)
                else:
                    pass
                    
                if lb==1:

                    features=librosa.core.stft(audio,n_fft=1024,hop_length=256,win_length=1024)
                    if features.shape[1]==87:
                        X_p.append(features)
                        Y_p.append(1)
                    else:
                        pass
                else:
                    pass

#                     features=librosa.core.stft(audio,n_fft=1024,hop_length=256,win_length=1024)
#                     if features.shape[1]==87:
#                         X_n.append(features)
#                         Y_n.append(0)
#                     else:
#                         pass
                start+=1
                count+=1
        except BaseException as e:
            pass

    X_p=np.array(X_p)
    #X_n=np.array(X_n)
    Y_p=np.array(Y_p)
    #Y_n=np.array(Y_n)

#     shuffle(X_p,Y_p)
#     shuffle(X_n,Y_n)

#     upto=min(len(Y_p),len(Y_n))

#     X_n=X_n[0:upto]
#     Y_n=Y_n[0:upto]

#     X=np.concatenate((X_p,X_n))
#     Y=np.concatenate((Y_p,Y_n))
    print(X_p.shape)
    np.save('./Custom/X_{filename}_p.npy'.format(filename=name),X_p)
    np.save('./Custom/Y_{filename}_p.npy'.format(filename=name),Y_p)


### Extracting both positive and negative speech data.
#### In actual practice I repeated this for a lot of files.

In [ ]:
for name in ['track42','track43','track44','track45']:
    annotations=pd.read_csv('{filename}.csv'.format(filename=name),header=None)

    X_p=[]
    X_n=[]
    Y_p=[]
    Y_n=[]
    test=[]
    count=0
    #len(annotations)
    for i in range(len(annotations)):
        try:

            print("Pass:%d"%i)
            _,start,end,lb=annotations.iloc[i]
            start,end,lb=int(start),int(end),int(lb)

            for j in range(abs(round((start-end))+1)):
                audio,sr=librosa.load('{filename}.mp3'.format(filename=name),offset=start,duration=1)
                if lb==1:

                    features=librosa.core.stft(audio,n_fft=1024,hop_length=256,win_length=1024)
                    if features.shape[1]==87:
                        X_p.append(features)
                        Y_p.append(1)
                    else:
                        pass
                else:
                    

                    features=librosa.core.stft(audio,n_fft=1024,hop_length=256,win_length=1024)
                    if features.shape[1]==87:
                        X_n.append(features)
                        Y_n.append(0)
                    else:
                        pass
                start+=1
                count+=1
        except BaseException as e:
            pass

    X_p=np.array(X_p)
    X_n=np.array(X_n)
    Y_p=np.array(Y_p)
    Y_n=np.array(Y_n)

    shuffle(X_p,Y_p)
    shuffle(X_n,Y_n)

    upto=min(len(Y_p),len(Y_n))

    X_n=X_n[0:upto]
    Y_n=Y_n[0:upto]

    X=np.concatenate((X_p,X_n))
    Y=np.concatenate((Y_p,Y_n))
    print(X.shape)
    np.save('X_{filename}.npy'.format(filename=name),X)
    np.save('Y_{filename}.npy'.format(filename=name),Y)


### Merging those numpy dumps to form dask array as we can't store that much arrays in low memory RAMs such as 8gb RAM.

In [2]:
def createDaskArray_x(darr,npylist):
    for arr_name in npylist:
        X=np.load('./custom/'+arr_name)
        X=np.expand_dims(np.expand_dims(X,axis=-1),axis=1)
        darr=da.concatenate([X,darr],axis=0)
        del X
    return darr
def createDaskArray_y(darr,npylist):
    for arr_name in npylist:
        X=np.load('./custom/'+arr_name)
        darr=da.concatenate([X,darr],axis=0)
        del X
    return darr
X=np.load('./custom/'+'X_track46_p.npy')
X=np.expand_dims(np.expand_dims(X,axis=-1),axis=1)
dax=da.from_array(X)
del X

Y=np.load('./custom/'+'Y_track46_p.npy')
day=da.from_array(Y)
del Y

array_dumps_x=['X_track69_p.npy','X_track70_p.npy','X_track71_p.npy','X_track72_p.npy']

array_dumps_y=['Y_track69_p.npy','Y_track70_p.npy','Y_track71_p.npy','Y_track72_p.npy']





daf1_x=createDaskArray_x(dax,array_dumps_x)
daf1_y=createDaskArray_y(day,array_dumps_y)

In [4]:
daf1_x.shape

(2023, 1, 513, 87, 1)

### Getting some negative samples from some other track in order to balance the negative samples in case of merging arrays only with positive samples.

In [5]:


for i in range(80,103):
    audio,sr=librosa.load('E:\\HVD Dataset\\ADHM.mp3',sr=22050,duration=1)
    features=librosa.core.stft(audio,n_fft=1024,hop_length=256,win_length=1024)
    if features.shape[1]==87:
        X_fil.append(features)
        Y_fil.append(0)

X_fil=np.array(X_fil)
Y_fil=np.array(Y_fil)
            
        
        

        

In [6]:
X_fil=np.expand_dims(np.expand_dims(X_fil,axis=-1),axis=1)

In [7]:
X_fil.shape

(23, 1, 513, 87, 1)

In [8]:
def createDaskArray_x(darr,npylist):
    for arr_name in npylist:
        X=np.load('./custom/'+arr_name)
        X=np.expand_dims(np.expand_dims(X,axis=-1),axis=1)
        darr=da.concatenate([X,darr],axis=0)
        del X
    return darr
def createDaskArray_y(darr,npylist):
    for arr_name in npylist:
        X=np.load('./custom/'+arr_name)
        darr=da.concatenate([X,darr],axis=0)
        del X
    return darr
X=np.load('./custom/'+'X_neg8.npy')
X=np.expand_dims(np.expand_dims(X,axis=-1),axis=1)
dax=da.from_array(X)
del X

Y=np.load('./custom/'+'Y_neg8.npy')
day=da.from_array(Y)
del Y

array_dumps_x=['X_neg9.npy']
array_dumps_y=['Y_neg9.npy']


daf2_x=createDaskArray_x(dax,array_dumps_x)
daf2_y=createDaskArray_y(day,array_dumps_y)



In [9]:
daf_c_x=da.concatenate([daf1_x,daf2_x],axis=0)
daf_c_y=da.concatenate([daf1_y,daf2_y],axis=0)
del daf1_x,daf2_x,daf1_y,daf2_y

In [10]:
daf_c_x.shape

(4023, 1, 513, 87, 1)

In [11]:
daf_c_x=da.concatenate([daf_c_x,X_fil],axis=0)
daf_c_y=da.concatenate([daf_c_y,Y_fil],axis=0)

In [12]:
daf_c_x.shape

(4046, 1, 513, 87, 1)

In [13]:
def createDaskArray_x(darr,npylist):
    for arr_name in npylist:
        X=np.load(arr_name)
        X=np.expand_dims(np.expand_dims(X,axis=-1),axis=1)
        darr=da.concatenate([X,darr],axis=0)
        del X
    return darr
def createDaskArray_y(darr,npylist):
    for arr_name in npylist:
        X=np.load(arr_name)
        darr=da.concatenate([X,darr],axis=0)
        del X
    return darr



X=np.load('X_track14.npy')
X=np.expand_dims(np.expand_dims(X,axis=-1),axis=1)
dax=da.from_array(X,chunks='auto')
del X

Y=np.load('Y_track14.npy')
day=da.from_array(Y,chunks='auto')
del Y

array_dumps_x=['X_track15.npy','X_track16.npy','X_track17.npy','X_track42.npy','X_track43.npy','X_track44.npy',
              'X_track45.npy']
array_dumps_y=['Y_track15.npy','Y_track16.npy','Y_track17.npy','Y_track42.npy','Y_track43.npy','Y_track44.npy',
              'Y_track45.npy']

dafv_x=createDaskArray_x(dax,array_dumps_x)
dafv_y=createDaskArray_y(day,array_dumps_y)

In [14]:
davf_x=da.concatenate([daf_c_x,dafv_x],axis=0)
davf_y=da.concatenate([daf_c_y,dafv_y],axis=0)

In [15]:
davf_y.shape

(8745,)

### Dumping the dask arrays as hdf5 arrays as hdf5 can store arrays as file in disk.

In [16]:

hdf5_store=h5py.File('./X_val1.hdf5','a')
results=hdf5_store.create_dataset("results",data=davf_x)
hdf5_store.close()




In [17]:
del daf_c_x
hdf5_store=h5py.File('./Y_val1.hdf5','a')
results=hdf5_store.create_dataset("results",data=davf_y)
hdf5_store.close()
del daf_c_y

### Creates negative samples from some tracks which contains only background sound.

In [ ]:
#Create the negative samples.
files_list=os.listdir('./custom/Background/')

def remove_files(lis):
    for filename in lis:
        os.remove('./custom/Background/'+filename)
X=[]
Y=[]
file_no=1
files_to_remove=[]
for filename in files_list:
    audio,sr=librosa.load('./custom/Background/'+filename)
    duration=int(librosa.core.get_duration(audio,sr))
    del audio,sr
    
    for i in range(duration):
        a,s=librosa.load('./custom/Background/'+filename,offset=i,duration=1)
        features=librosa.core.stft(a,n_fft=1024,hop_length=256,win_length=1024)
        if features.shape[1]==87:
            X.append(features)
            Y.append(0)
        
        if len(X)==6000:
            X=np.array(X)
            Y=np.array(Y)
            np.save(f'custom/X_neg{file_no}.npy',X)
            np.save(f'custom/Y_neg{file_no}.npy',Y)
            print(f'Created X_neg{file_no}.npy and Y_neg{file_no}.npy')
            #print(files_to_remove)
            remove_files(files_to_remove)
            files_to_remove=[]
            X=[]
            Y=[]
            
            file_no+=1
    files_to_remove.append(filename)
        
            
        
        